## Quantify hexapod motion fails
### This runs multiple images

Craig Lage  25-Jul-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.utils.simonyi.mountData import MountData

In [ ]:
client = makeEfdClient()
instrument = 'LSSTCam'
butler = butlerUtils.makeDefaultButler(instrument)

In [ ]:
def HexapodChanges(mountData: MountData) -> tuple[float, float]:
    """Calculate the image drift associated with the hexapod motions
    for a given exposure.
    Parameters
    ----------
    mountData : MountData
        The EFD data associated with the exposure
    Returns
    -------
    tuple[float, float, float, float, float,float, float, float, float, float]
        The hexapod changes during the exposures
    """

    # For each hexapod motion, fit a line and determine the slope
    changes = np.zeros([10])
    camhex = mountData.camhexData
    times = camhex['private_kafkaStamp']
    times -= times.iloc[0]
    for i in range(5): 
        dat = camhex[f'position{i}']
        fit = np.polyfit(times, dat, 1)
        change = fit[0] * times.iloc[-1]
        changes[i] = change

    m2hex = mountData.m2hexData
    times = m2hex['private_kafkaStamp']
    times -= times.iloc[0]
    for i in range(5):
        dat = m2hex[f'position{i}']
        fit = np.polyfit(times, dat, 1)
        change = fit[0] * times.iloc[-1]
        changes[i + 5] = change
    return changes

In [ ]:
startDay = 20251209
endDay = 20251211

hex_changes = {}
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science', 'acq', 'engtest']:
            continue
        try:
            (mountErrors, mountData) = calculateMountErrors(record, client)
            changes = HexapodChanges(mountData)
            hex_changes[record.id] = changes
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)


In [ ]:
fig, axes = plt.subplots(2,5,figsize=(12,5))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.suptitle("Hexapod motions while exposing, V20 vs V22")
names = [['CamX', 'CamY', 'CamZ', 'CamRx', 'CamRy'],
         ['M2X', 'M2Y', 'M2Z', 'M2Rx', 'M2Ry']]
for dayObs in [20251209, 20251210, 20251211]:
    counter = 0
    changes = np.zeros([5,2,500])
    for key in hex_changes.keys():
        if int(key / 1E5) == dayObs:
            for j in range(2):
                for i in range(5):
                    index = i + 5 * j
                    changes[i,j, counter] = hex_changes[key][index]
            counter += 1
        else:
            continue
    changes = np.delete(changes, np.arange(counter, 500), axis=2)
    for j in range(2):
        for i in range(5):
            axes[j][i].hist(changes[i,j,:], bins=20, histtype='step', label=dayObs)
            axes[j][i].set_title(names[j][i])
    axes[1][4].legend(loc='upper right', bbox_to_anchor=(1.8, 0.8))
plt.savefig(f"/home/c/cslage/u/Hexapods/data/Hexapod_Motions_while_Exposing_{startDay}-{endDay}.png")

In [ ]:
fig, axes = plt.subplots(2,5,figsize=(12,5))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.suptitle("Hexapod motions while exposing, V20 vs V22")
names = [['CamX', 'CamY', 'CamZ', 'CamRx', 'CamRy'],
         ['M2X', 'M2Y', 'M2Z', 'M2Rx', 'M2Ry']]
for dayObs in [20251214, 20251215]:
    counter = 0
    changes = np.zeros([5,2,500])
    for key in hex_changes.keys():
        if int(key / 1E5) == dayObs:
            for j in range(2):
                for i in range(5):
                    index = i + 5 * j
                    changes[i,j, counter] = hex_changes[key][index]
            counter += 1
        else:
            continue
    changes = np.delete(changes, np.arange(counter, 500), axis=2)
    for j in range(2):
        for i in range(5):
            axes[j][i].hist(changes[i,j,:], bins=20, histtype='step', label=dayObs)
            axes[j][i].set_title(names[j][i])
    axes[1][4].legend(loc='upper right', bbox_to_anchor=(1.8, 0.8))
    axes[1][2].set_xlim(-0.5, 0.5)
plt.savefig(f"/home/c/cslage/u/Hexapods/data/Hexapod_Motions_while_Exposing_{startDay}-{endDay}.png")